# 코드 개요
1. 크롤링 사이트 : 네이버쇼핑 - 네이버 페이탭
2. 크롤링 데이터 : 상품명, url
3. 1페이지부터 20페이지까지 순회하며 url 및 상품명 수집

## 주요 모듈 import

In [2]:
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from urllib.parse import quote
import chromedriver_autoinstaller
from bs4 import BeautifulSoup
import re
import pandas as pd
import time

## depth2(계절가전), depth3(에어컨) 수집

In [3]:
cat_name = ['계절가전','에어컨']
    # 파일 저장에 사용
urls = ['https://search.shopping.naver.com/search/category/100000581','https://search.shopping.naver.com/search/category/100000620#']
    # depth2와 depth3의 페이지 링크 포함

In [4]:
# 20페이지까지 상품명, url 수집
for i in range(2) :
            # range(2) 대신 range(len(urls)) 사용해도 무방 <= url 리스트 안에 있는 url을 순회하며 페이지 크롤링 해오는 게 목적인 반복문
    print(f"{'-'*20} {cat_name[i]} {'-'*20}")
            # 현재 어떤 depth에서 수집하고 있는지 정보 표시
    
    chrome_options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()), options = chrome_options) # 크롬드라이버 자동 인스톨, 안되면 직접 설치 필요
    #driver = webdriver.Chrome(service = Service('C:/Users/user/Desktop/작업용 폴더/chromedriver-win64/chromedriver-win64/chromedriver.exe'), options = chrome_options)
    driver.get(urls[i]) 
            # 계절가전, 에어컨 url 접근

    time.sleep(0.5)
    
    # 네이버 페이 클릭
    driver.find_element(By.XPATH, '//*[@id="content"]/div[1]/div[1]/ul/li[3]/a').click()
                                        # [네이버페이] 탭의 XPATH 값
    
    time.sleep(0.5)
    page_nums = 1
    prod_df = pd.DataFrame(columns = ['상품명','url'])
                # 상품명, url 두 열을 가지고 있는 데이터 프레임 완성
    while 1 :
            # while 1(True) : 무한 반복문 처리
        if page_nums == 21 :
            # 20페이지까지만 크롤링
            break
            
        SCROLL_PAUSE_SEC = 1

        # 스크롤 높이 가져옴
        last_height = driver.execute_script("return document.body.scrollHeight")

        while 1:
            # 끝까지 스크롤 다운 - 네이버 쇼핑 페이지가 스크롤을 내려야 추가정보를 가져오는 형태로 구성되어 있음
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            ''' 
            execute_script("스크립트", 요소) : 해당 페이지에 스크립트를 만들 때 사용.
                                                요소가 필수 파라미터는 아니나, 요소가 있으면 요소에 스크립트가 실행되고 없으면 전체페이지에 스크립트가 실행
            '''

            # 1초 대기
            time.sleep(SCROLL_PAUSE_SEC)

            # 스크롤 다운 후 스크롤 높이 다시 가져옴
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
                    # 더이상 스크롤이 내려가지 않을때까지 해당 루틴 반복 & 반복문 중지
            last_height = new_height
        time.sleep(1)
        soup = BeautifulSoup(driver.page_source,'html.parser')

        prod_list = soup.find_all('div',class_= 'product_title__Mmw2K')
                                                    # 페이지에 있는 상품에 대한 정보 리스트 추출
        for prod in prod_list :
            url = str(prod).split('href="')[1].split()[0]
                    # url 파서 중에서 'href=' 이후에 있는 부분만 추출
            prod_nm = str(prod).split('title=')[1].split('>')[0].replace('"','')
                    # 상품명 파서 중에서 1) 'title='이후에 있는 부분 추출 2) '>' 앞에 있는 부분만 추출 3) 남은 "부분 제거
            new_df = pd.DataFrame([[prod_nm,url]],columns = ['상품명','url'])
                    # 데이터프레임에 상품명과 url 값 입력
            prod_df = pd.concat([prod_df,new_df])

        
        page_nums += 1
        if page_nums == 21 :
            break
            # 20페이지까지만 해당 코드 반복
        elif page_nums-1 == 1 :
            driver.find_element(By.XPATH,'//*[@id="content"]/div[1]/div[4]/a').click()
                                                # 다음 클릭
        else :
            driver.find_element(By.XPATH,'//*[@id="content"]/div[1]/div[4]/a[2]').click()
                                                # 다음 클릭
        time.sleep(1)
        print('page_nums:', page_nums-1, '수집 개수:',len(prod_df))

    prod_df = prod_df.reset_index(drop = True)
    file_name = f'{cat_name[i]}(네이버페이,800개).csv'
    prod_df.to_csv(file_name, index = False, encoding = 'utf-8-sig')

-------------------- 계절가전 --------------------
page_nums: 1 수집 개수: 40
page_nums: 2 수집 개수: 80
page_nums: 3 수집 개수: 120
page_nums: 4 수집 개수: 160
page_nums: 5 수집 개수: 200
page_nums: 6 수집 개수: 240
page_nums: 7 수집 개수: 280
page_nums: 8 수집 개수: 320
page_nums: 9 수집 개수: 360
page_nums: 10 수집 개수: 400
page_nums: 11 수집 개수: 440
page_nums: 12 수집 개수: 480
page_nums: 13 수집 개수: 520
page_nums: 14 수집 개수: 560
page_nums: 15 수집 개수: 600
page_nums: 16 수집 개수: 640
page_nums: 17 수집 개수: 680
page_nums: 18 수집 개수: 720
page_nums: 19 수집 개수: 760
-------------------- 에어컨 --------------------
page_nums: 1 수집 개수: 40
page_nums: 2 수집 개수: 80
page_nums: 3 수집 개수: 120
page_nums: 4 수집 개수: 160
page_nums: 5 수집 개수: 200
page_nums: 6 수집 개수: 240
page_nums: 7 수집 개수: 280
page_nums: 8 수집 개수: 320
page_nums: 9 수집 개수: 360
page_nums: 10 수집 개수: 400
page_nums: 11 수집 개수: 440
page_nums: 12 수집 개수: 480
page_nums: 13 수집 개수: 520
page_nums: 14 수집 개수: 560
page_nums: 15 수집 개수: 600
page_nums: 16 수집 개수: 640
page_nums: 17 수집 개수: 680
page_nums: 18 수집 개수: 720
page

## depth4 수집

In [5]:
def page_crawl():
    page_nums = 1
    prod_df = pd.DataFrame( columns = ['상품명', 'url'])

    while 1: 
        if page_nums == 21:
            break 
                # 20페이지까지만 크롤링 코드 반복

        SCROLL_PAUSE_SEC = 1

        # 스크롤 높이 
        last_height = driver.execute_script("return document.body.scrollHeight")

        while 1:
            # 끝까지 스크롤 다운
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            time.sleep(SCROLL_PAUSE_SEC)

            # 스크롤 다운 후 스크롤 높이 다시 가져옴
            new_height = driver.execute_script("return document.body.scrollHeight")

            if new_height == last_height:
                break
            last_height = new_height
        time.sleep(1)

        soup = BeautifulSoup(driver.page_source, 'html.parser')

        prod_list = soup.find_all('div', class_='product_title_Mmw2K')
        for prod in prod_list:
            url = str(prod).split('href="')[1].split()[0]
            prod_nm = str(prod).split('title=')[1].split('>')[0].replace('"','')
            new_df = pd.DataFrame([[prod_nm, url]], columns = ['상품명', 'url'])
            prod_df = pd.concat([prod_df, new_df])

        page_nums += 1
        if page_nums == 21 :
            break
        elif page_nums-1 == 1 :
            driver.find_element(By.XPATH, '//*[@id="content"]/div[1]/div[4]/a').click()
        else :
            driver.find_element(By.XPATH,'//*[@id="content"]/div[1]/div[4]/a[2]').click()
        time.sleep(1)

        print('page nums:', page_nums-1, '수집개수 :', len(prod_df))

    return prod_df

        

In [ ]:
# //*[@id="container"]/div/div[2]/div[1]/div[1]/div[2]/div/ul
# //*[@id="container"]/div/div[2]/div[1]/div[1]/div[2]/div/ul/li[1]/a/span

In [7]:
import time
url = 'https://search.shopping.naver.com/search/category/100000620?catId=50001421&origQuery&pagingIndex=1&pagingSize=40&productSet=total&query&sort=rel&timestamp=&viewType=list'
chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()), options = chrome_options) # 크롬드라이버 자동 인스톨, 안되면 직접 설치 필요
# driver = webdriver.Chrome(service = Service('C:/Users/user/Desktop/작업용 폴더/chromedriver-win64/chromedriver-win64/chromedriver.exe'), options = chrome_options)
driver.get(url)
cat_list = driver.find_element(By.XPATH,'//*[@id="container"]/div/div[2]/div[1]/div[1]/div[2]/div/ul').text.split('\n')
                                    # 카테고리 리스트 추출
for cat in driver.find_element(By.XPATH,'//*[@id="container"]/div/div[2]/div[1]/div[1]/div[2]/div/ul').text.split('\n') :
    print(f"{'-'*20} {cat} {'-'*20}")
    cat_idx = cat_list.index(cat)+1
                    # cat_list에서 현재 반복중인 부분의 순서
                        ## cat_list에서 0번째이면 cat_idx = 1
    driver.get(url)
    time.sleep(2)
    cat_xpath = f'//*[@id="container"]/div/div[2]/div[1]/div[1]/div[2]/div/ul/li[{cat_idx}]/a' # 카테고리 클릭 
                        # cat_idx = 1 이므로 li[1]번째 요소를 클릭
    driver.find_element(By.XPATH, cat_xpath).click() 
    time.sleep(2)
    driver.find_element(By.XPATH, '//*[@id="content"]/div[1]/div[1]/ul/li[3]/a').click()
                        # 네이버페이 탭 클릭
    # driver.find_element(By.XPATH, '//*[@id="content"]/div[1]/div[1]/div/div[2]/div[1]/a').click()
    # time.sleep(0.3)
    # driver.find_element(By.XPATH, '//*[@id="content"]/div[1]/div[1]/div/div[2]/div[1]/div/ul/li[7]/a').click()
    # time.sleep(1)
    # for url_num in range(1,len(driver.find_element(By.XPATH,'//*[@id="content"]/div[1]/div[1]/div/div[2]/div[1]/div/div[2]').text.split('\n'))+1) :
    #     checkbox_xpath = f'//*[@id="content"]/div[1]/div[1]/div/div[2]/div[1]/div/div[2]/ul/li[{str(url_num)}]'
    #     driver.find_element(By.XPATH, checkbox_xpath).click()
    #     time.sleep(0.5)
    # driver.find_element(By.XPATH,'//*[@id="content"]/div[1]/div[1]/div/div[2]/div[1]/div/div[3]/a[1]').click()
    # time.sleep(0.5)
    # cat_xpath = f'//*[@id="container"]/div/div[2]/div[1]/div[1]/div[2]/div/ul/li[{cat_idx}]/a'
    # driver.find_element(By.XPATH, cat_xpath).click()
    time.sleep(0.5)
    prod_df = page_crawl()
        # 해당 카테고리 내 20페이지까지 상품 크롤링
    file_name = f'{cat}_URL(네이버페이, 800개).csv'
    prod_df.to_csv(file_name,index = False, encoding = 'utf-8-sig')
    

-------------------- 벽걸이형 --------------------
page nums: 1 수집개수 : 0
page nums: 2 수집개수 : 0
page nums: 3 수집개수 : 0
page nums: 4 수집개수 : 0
page nums: 5 수집개수 : 0
page nums: 6 수집개수 : 0
page nums: 7 수집개수 : 0
page nums: 8 수집개수 : 0
page nums: 9 수집개수 : 0
page nums: 10 수집개수 : 0
page nums: 11 수집개수 : 0
page nums: 12 수집개수 : 0
page nums: 13 수집개수 : 0
page nums: 14 수집개수 : 0
page nums: 15 수집개수 : 0
page nums: 16 수집개수 : 0
page nums: 17 수집개수 : 0
page nums: 18 수집개수 : 0
page nums: 19 수집개수 : 0
-------------------- 스탠드형 --------------------
page nums: 1 수집개수 : 0
page nums: 2 수집개수 : 0
page nums: 3 수집개수 : 0
page nums: 4 수집개수 : 0
page nums: 5 수집개수 : 0
page nums: 6 수집개수 : 0
page nums: 7 수집개수 : 0
page nums: 8 수집개수 : 0
page nums: 9 수집개수 : 0
page nums: 10 수집개수 : 0
page nums: 11 수집개수 : 0
page nums: 12 수집개수 : 0
page nums: 13 수집개수 : 0
page nums: 14 수집개수 : 0
page nums: 15 수집개수 : 0
page nums: 16 수집개수 : 0
page nums: 17 수집개수 : 0
page nums: 18 수집개수 : 0
page nums: 19 수집개수 : 0
-------------------- 멀티형 --------------------
page